In [ ]:
from braindecode.datasets import TUHAbnormal

TUHAbnormal_PATH = '/home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0'
N_JOBS = 8  # specify the number of jobs for loading and windowing


tuh = TUHAbnormal(
    path=TUHAbnormal_PATH,
    recording_ids=None,
    target_name=('report','pathological','gender','age'),
    preload=False,
    add_physician_reports=True,
    n_jobs=N_JOBS, 
)

In [5]:
print("length of dataset : ", len(tuh))

x, y = tuh[-1]
print('y:', y)

length of dataset :  1035794884
y: ['CLINICAL HISTORY:  A 21-year-old female with multiple concussions, most recently October 6, with headaches and intermittent difficulty with concentration.\nMEDICATIONS:  Elavil, albuterol, Naprosyn.\nINTRODUCTION:  Digital video EEG was performed in the lab using standard 10-20 system of electrode placement with 1 channel EKG.  Hyperventilation and photic stimulation were completed.  This is an awake and asleep record.  It is technically satisfactory with impedances under 5 kilohms.\nDESCRIPTION OF THE RECORD:  In wakefulness, there is a 10 Hz alpha rhythm which is symmetric and reactive to eye opening.  Alpha squeak was noted.  Brief drowsiness is characterized by an increase in beta, but sleep was not sustained.\nHyperventilation produced a small increase in amplitude of the background.  Photic stimulation elicited scant driving.\nHeart rate: 55 bpm.\nIMPRESSION:  Normal EEG in wakefulness and brief drowsiness.\nCLINICAL CORRELATION:  No focal or 

In [6]:
tuh_subjects = tuh.split('subject')
len(tuh_subjects)
key_list = list(tuh_subjects.keys())

In [1]:
import pandas as pd

In [27]:
"""
df = pd.DataFrame(columns=('report', 'pathological', 'gender', 'age'))
for i in range(len(key_list)):
   df.loc[i] = tuh_subjects[key_list[i]][0][1]
df.to_csv("report_df.csv")
"""

In [2]:
report_df = pd.read_csv("report_df.csv")

In [5]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")
report_embeddings = []
for r in report_df['report']:
        print(r)
        desc_tokenized = bert_tokenizer(r, return_tensors="pt", max_length=512, truncation=True, padding='max_length')
        outputs = bert_model(**desc_tokenized)

        report_embeddings.append(outputs.to_tuple()[0][0][0].detach().numpy().tolist())

report_embeddings = np.array(report_embeddings)

